In [11]:
import random

class Navire:
    def __init__(self, type_navire, sante_base, degats_base, precision_base, esquive_base):
        self.type_navire = type_navire
        self.sante_base = sante_base
        self.degats_base = degats_base
        self.precision_base = precision_base
        self.esquive_base = esquive_base
        self.destructions = 0  

    def calculer_stats_effectives(self, moral, discipline, bonus_amiral):
        self.sante = self.sante_base * (1 + moral / 100)
        self.degats = self.degats_base * (1 + discipline / 100) + bonus_amiral['degats']
        self.precision = self.precision_base + bonus_amiral['precision']
        self.esquive = self.esquive_base + bonus_amiral['esquive']

def est_endommage(navire):
    return navire.sante <= navire.sante_base * 0.5

def simulate_battle(ships_fleet1, ships_fleet2, admiral1, admiral2, max_turns=30):
    index1, index2, turn_count = 0, 0, 0
    while ships_fleet1 and ships_fleet2 and turn_count < max_turns:
        initiative_fleet1 = admiral1['initiative'] + random.randint(1, 6)
        initiative_fleet2 = admiral2['initiative'] + random.randint(1, 6)

        if initiative_fleet1 >= initiative_fleet2:
            first_attacker, second_attacker = ships_fleet1, ships_fleet2
            first_index, second_index = index1, index2
        else:
            first_attacker, second_attacker = ships_fleet2, ships_fleet1
            first_index, second_index = index2, index1

        # Attaque de la première flotte
        if first_attacker:
            attacking_ship = first_attacker[first_index]
            defending_ship = second_attacker[second_index]
            if random.uniform(0, 100) <= attacking_ship.precision:
                if random.uniform(0, 100) > defending_ship.esquive:
                    defending_ship.sante -= attacking_ship.degats
                    if defending_ship.sante <= 0:
                        attacking_ship.destructions += 1

        # Attaque de la seconde flotte
        if second_attacker:
            attacking_ship = second_attacker[second_index]
            defending_ship = first_attacker[first_index]
            if random.uniform(0, 100) <= attacking_ship.precision:
                if random.uniform(0, 100) > defending_ship.esquive:
                    defending_ship.sante -= attacking_ship.degats
                    if defending_ship.sante <= 0:
                        attacking_ship.destructions += 1

       
        if ships_fleet1 and ships_fleet1[first_index].sante <= 0:
            ships_fleet1.pop(first_index)
            if first_index >= len(ships_fleet1) and ships_fleet1:
                first_index = 0
        else:
            first_index = (first_index + 1) % len(ships_fleet1) if ships_fleet1 else 0

        if ships_fleet2 and ships_fleet2[second_index].sante <= 0:
            ships_fleet2.pop(second_index)
            if second_index >= len(ships_fleet2) and ships_fleet2:
                second_index = 0
        else:
            second_index = (second_index + 1) % len(ships_fleet2) if ships_fleet2 else 0

        index1, index2 = (first_index, second_index) if initiative_fleet1 >= initiative_fleet2 else (second_index, first_index)
        turn_count += 1

    return {
        "fleet1_survivors": [ship.type_navire for ship in ships_fleet1],
        "fleet2_survivors": [ship.type_navire for ship in ships_fleet2],
        "turns": turn_count
    }

def compter_navires(flotte):
    comptage = {}
    for navire in flotte:
        if navire.type_navire not in comptage:
            comptage[navire.type_navire] = 0
        comptage[navire.type_navire] += 1
    return comptage

def principal():
    types_navires = {
        "Corvette": Navire("Corvette", 100, 50, 90, 60),
        "Destroyer": Navire("Destroyer", 150, 70, 85, 40),
        "Croiseur": Navire("Croiseur", 200, 90, 70, 20),
        "Cuirassé": Navire("Cuirassé", 300, 120, 60, 0)
    }

    # Entrées utilisateur pour la flotte 1
    compte_navires_flotte1 = {type_navire: int(input(f"Nombre de {type_navire}s dans la Flotte 1: ")) for type_navire in types_navires}
    bonus_moral_flotte1 = float(input("Entrez le bonus de moral pour la Flotte 1: "))
    bonus_discipline_flotte1 = float(input("Entrez le bonus de discipline pour la Flotte 1: "))
    bonus_amiral_flotte1 = {
        'degats': float(input("Entrez le bonus de dégâts de l'amiral pour la Flotte 1: ")),
        'precision': float(input("Entrez le bonus de précision de l'amiral pour la Flotte 1: ")),
        'esquive': float(input("Entrez le bonus d'esquive de l'amiral pour la Flotte 1: ")),
        'initiative': float(input("Entrez l'initiative de l'amiral pour la Flotte 1: "))
    }

    # Entrées utilisateur pour la flotte 2
    compte_navires_flotte2 = {type_navire: int(input(f"Nombre de {type_navire}s dans la Flotte 2: ")) for type_navire in types_navires}
    bonus_moral_flotte2 = float(input("Entrez le bonus de moral pour la Flotte 2: "))
    bonus_discipline_flotte2 = float(input("Entrez le bonus de discipline pour la Flotte 2: "))
    bonus_amiral_flotte2 = {
        'degats': float(input("Entrez le bonus de dégâts de l'amiral pour la Flotte 2: ")),
        'precision': float(input("Entrez le bonus de précision de l'amiral pour la Flotte 2: ")),
        'esquive': float(input("Entrez le bonus d'esquive de l'amiral pour la Flotte 2: ")),
        'initiative': float(input("Entrez l'initiative de l'amiral pour la Flotte 2: "))
    }

    # Création des flottes
    flotte1 = [types_navires[type_navire] for type_navire, compte in compte_navires_flotte1.items() for _ in range(compte)]
    for navire in flotte1:
        navire.calculer_stats_effectives(bonus_moral_flotte1, bonus_discipline_flotte1, bonus_amiral_flotte1)

    flotte2 = [types_navires[type_navire] for type_navire, compte in compte_navires_flotte2.items() for _ in range(compte)]
    for navire in flotte2:
        navire.calculer_stats_effectives(bonus_moral_flotte2, bonus_discipline_flotte2, bonus_amiral_flotte2)

    # Compter les navires dans chaque flotte avant la bataille
    comptage_initial_flotte1 = compter_navires(flotte1)
    comptage_initial_flotte2 = compter_navires(flotte2)

    # Simulation de la bataille
    resultats_bataille = simulate_battle(flotte1, flotte2, bonus_amiral_flotte1, bonus_amiral_flotte2)

    # Compter les survivants après la bataille
    comptage_survivants_flotte1 = compter_navires(flotte1)
    comptage_survivants_flotte2 = compter_navires(flotte2)

    # Afficher les résultats pour la Flotte 1
    print("Survivants de la Flotte 1:")
    for type_navire in types_navires:
        endommages = sum(est_endommage(navire) for navire in flotte1 if navire.type_navire == type_navire)
        operationnels = comptage_survivants_flotte1.get(type_navire, 0) - endommages
        perdus = comptage_initial_flotte1[type_navire] - operationnels - endommages
        print(f"{type_navire}: Opérationnelle - {operationnels}, Perdus - {perdus}, Endommagée - {endommages}")

    # Afficher les résultats pour la Flotte 2
    print("\nSurvivants de la Flotte 2:")
    for type_navire in types_navires:
        endommages = sum(est_endommage(navire) for navire in flotte2 if navire.type_navire == type_navire)
        operationnels = comptage_survivants_flotte2.get(type_navire, 0) - endommages
        perdus = comptage_initial_flotte2[type_navire] - operationnels - endommages
        print(f"{type_navire}: Opérationnelle - {operationnels}, Perdus - {perdus}, Endommagée - {endommages}")

    # Déterminer le navire avec le plus de destructions dans chaque flotte
    max_destructions_fleet1 = max(flotte1, key=lambda navire: navire.destructions, default=None)
    max_destructions_fleet2 = max(flotte2, key=lambda navire: navire.destructions, default=None)

    if max_destructions_fleet1:
        print(f"\nNavire le plus destructeur dans la Flotte 1: {max_destructions_fleet1.type_navire} avec {max_destructions_fleet1.destructions} destructions")
    else:
        print("Aucun navire destructeur dans la Flotte 1")

    if max_destructions_fleet2:
        print(f"Navire le plus destructeur dans la Flotte 2: {max_destructions_fleet2.type_navire} avec {max_destructions_fleet2.destructions} destructions")
    else:
        print("Aucun navire destructeur dans la Flotte 2")

    print(f"\nNombre de tours: {resultats_bataille['turns']}")

# Lancer la fonction principale
principal()




Survivants de la Flotte 1:
Corvette: Opérationnelle - 0, Perdus - 10, Endommagée - 2
Destroyer: Opérationnelle - 0, Perdus - 11, Endommagée - 4
Croiseur: Opérationnelle - 0, Perdus - 1, Endommagée - 411
Cuirassé: Opérationnelle - 4, Perdus - 0, Endommagée - 0

Survivants de la Flotte 2:
Corvette: Opérationnelle - 0, Perdus - 28, Endommagée - 72
Destroyer: Opérationnelle - 0, Perdus - 0, Endommagée - 12
Croiseur: Opérationnelle - 0, Perdus - 0, Endommagée - 12
Cuirassé: Opérationnelle - 50, Perdus - 0, Endommagée - 0

Navire le plus destructeur dans la Flotte 1: Corvette avec 13 destructions
Navire le plus destructeur dans la Flotte 2: Corvette avec 13 destructions

Nombre de tours: 30
